# PyTorch Tensor Basics
This notebook demonstrates basic tensor creation and operations using PyTorch.

In [44]:
import torch

## Tensor Operations- Understanding tensors

In [45]:
tensor0d = torch.tensor(1.0)
tensor1d = torch.tensor([1, 2, 3])
# create a 2D tensor from a nested Python list
tensor2d = torch.tensor([[1, 2, 3],
                         [4, 5, 6]])
# create a 3D tensor from a nested Python list
tensor3d = torch.tensor([[[1, 2], [3, 4]], [[5, 6], [7, 8]]])

In [46]:
print(tensor2d)
print(tensor1d.dtype)
print(tensor2d.shape)
print(tensor2d.reshape(3, 2))
print(tensor2d.T)
print(tensor2d.matmul(tensor2d.T))
print(tensor2d @ tensor2d.T)

tensor([[1, 2, 3],
        [4, 5, 6]])
torch.int64
torch.Size([2, 3])
tensor([[1, 2],
        [3, 4],
        [5, 6]])
tensor([[1, 4],
        [2, 5],
        [3, 6]])
tensor([[14, 32],
        [32, 77]])
tensor([[14, 32],
        [32, 77]])


## Computation graphs
A computation graph is a directed graph that allows us to express and visualize mathematical operations.

In [47]:
import torch.nn.functional as F

y = torch.tensor([1.0])  # true label
x1 = torch.tensor([1.1]) # input feature
w1 = torch.tensor([2.2]) # weight parameter
b = torch.tensor([0.0])  # bias unit
z = x1 * w1 + b          # net input
a = torch.sigmoid(z)     # activation & output
loss = F.binary_cross_entropy(a, y)  # compute loss
print(loss)

tensor(0.0852)


## Automatic differentiation made easy

By tracking every operation performed on tensors, PyTorch’s autograd engine constructs a computational graph in the background. Then, calling the grad function, we can compute the gradient of the loss with respect to model parameter w1 as follows:

In [48]:
import torch.nn.functional as F
from torch.autograd import grad

y = torch.tensor([1.0])
x1 = torch.tensor([1.1])
w1 = torch.tensor([2.2], requires_grad=True)
b = torch.tensor([0.0], requires_grad=True)

z = x1 * w1 + b
a = torch.sigmoid(z)

loss = F.binary_cross_entropy(a, y)

grad_L_w1 = grad(loss, w1, retain_graph=True)
grad_L_b = grad(loss, b, retain_graph=True)
print(grad_L_w1)
print(grad_L_b)

(tensor([-0.0898]),)
(tensor([-0.0817]),)


### Automatically storing the gradient of all the leaf nodes

In [49]:
loss.backward()

print(w1.grad)
print(b.grad)

tensor([-0.0898])
tensor([-0.0817])


# Implementing multi layer neural networks


In [52]:
class NeuralNetwork(torch.nn.Module):
    def __init__(self, num_inputs, num_outputs):
        super().__init__()

        self.layers = torch.nn.Sequential(

            # 1st hidden layer
            torch.nn.Linear(num_inputs, 30),
            torch.nn.ReLU(),

            # 2nd hidden layer
            torch.nn.Linear(30, 20),
            torch.nn.ReLU(),

            # output layer
            torch.nn.Linear(20, num_outputs),
        )

    def forward(self, x):
        logits = self.layers(x)
        return logits

model = NeuralNetwork(50, 3)
print(model)
num_params = sum(
    p.numel() for p in model.parameters() if p.requires_grad
)
print("Total number of trainable model parameters:", num_params)
print(model.layers[0].weight)
# dimension of a layer
print(model.layers[0].weight.shape)

NeuralNetwork(
  (layers): Sequential(
    (0): Linear(in_features=50, out_features=30, bias=True)
    (1): ReLU()
    (2): Linear(in_features=30, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=3, bias=True)
  )
)
Total number of trainable model parameters: 2213
Parameter containing:
tensor([[ 0.1111, -0.1394, -0.0535,  ...,  0.0168, -0.0310,  0.0212],
        [-0.1181, -0.1100, -0.0147,  ...,  0.0097, -0.0307,  0.1348],
        [ 0.0422, -0.0288, -0.0292,  ...,  0.1062, -0.0263,  0.0925],
        ...,
        [ 0.1049, -0.0061,  0.1247,  ...,  0.0097, -0.0399,  0.0880],
        [-0.1123, -0.0133, -0.0422,  ..., -0.0186,  0.1366,  0.1368],
        [-0.0490, -0.0387, -0.0452,  ...,  0.1263, -0.0174,  0.0117]],
       requires_grad=True)
torch.Size([30, 50])


## Seeding random number generator

In [ ]:
torch.manual_seed(123)

model = NeuralNetwork(50, 3)
print(model.layers[0].weight)